## <class 'function'> 和 <class 'method'>

与类和实例无绑定关系的 function 都属于 `函数（function）`，否则属于 `方法（method）`。

| 访问方式 | 函数类型 | 返回类型 | 绑定状态 | 说明 |
|:---------|:---------|:---------|:---------|:------|
| `standalone_func` | 独立函数 | `<class 'function'>` | 无绑定 | 模块级函数，不属于任何类 |
| `Class.method` | 实例方法 | `<class 'function'>` | 未绑定 | 从类访问，返回原始函数对象 |
| `instance.method` | 实例方法 | `<class 'method'>` | 绑定到实例 | 自动将实例作为第一个参数 |
| `Class.static_method` | 静态方法 | `<class 'function'>` | 无绑定 | 与类和实例都无关联 |
| `instance.static_method` | 静态方法 | `<class 'function'>` | 无绑定 | 与类和实例都无关联 |
| `Class.class_method` | 类方法 | `<class 'method'>` | 绑定到类 | 自动将类作为第一个参数 |
| `instance.class_method` | 类方法 | `<class 'method'>` | 绑定到类 | 自动将类作为第一个参数 |
| `lambda x: x` | Lambda函数 | `<class 'function'>` | 无绑定 | 匿名函数，独立存在 |
| `len`, `print`, `max` | 内置函数 | `<class 'builtin_function_or_method'>` | 特殊 | C语言实现的内置函数 |

| 特性 | `<class 'function'>` | `<class 'method'>` |
|:-----|:---------------------|:-------------------|
| 调用方式 | 直接通过函数名调用 | 通过`对象.方法名`调用 |
| 参数传递 | 需要显式传递所有参数 | 自动传递绑定对象作为第一个参数 |
| 独立性 | 完全独立，不依赖任何对象 | 依赖于绑定的对象（类或实例） |

## <class 'function'> 都是描述符即实现了 `__get__`
`<class 'function'>` 是在 CPython 中的 C 代码中实现的，其 Python 等价实现：
```python
class Function:
    def __init__(self, func_code, func_name):
        self.__code__ = func_code
        self.__name__ = func_name
    
    def __get__(self, instance, owner=None):
        """函数的描述符协议实现"""
        if instance is None:
            # 从类访问：返回未绑定的函数
            return self
        else:
            # 从实例访问：创建绑定方法
            import types
            return types.MethodType(self, instance)
    
    def __call__(self, *args, **kwargs):
        # 函数调用逻辑
        pass

class TestClass:
    def my_method(self, x):
        return f"处理: {x}"

obj = TestClass()

# 手动调用函数的 __get__ 方法
func = TestClass.my_method  # 这是一个 function 对象
print(f"原始函数类型: {type(func)}")  # <class 'function'>

# 手动调用 __get__ 模拟从类访问
result1 = func.__get__(None, TestClass)
print(f"从类访问结果: {type(result1)}")  # <class 'function'>

# 手动调用 __get__ 模拟从实例访问  
result2 = func.__get__(obj, TestClass)
print(f"从实例访问结果: {type(result2)}")  # <class 'method'>

# 验证绑定方法的属性
print(f"绑定的实例: {result2.__self__ is obj}")      # True
print(f"原始函数: {result2.__func__ is func}")        # True
```


## 将 `function` 绑定为 `method`
使用 `types.MethodType` 可以将 `<class 'function'>` 变成了 `<class 'method'>`。
### 基本转换示例
```python
import types

def standalone_function(self, x):
    return f"处理 {x} 来自 {self}"

class MyClass:
    def __init__(self, name):
        self.name = name

obj = MyClass("测试对象")

print("=== 转换前后的类型对比 ===")
print(f"原始函数类型: {type(standalone_function)}")  # <class 'function'>

# 使用 MethodType 绑定到实例
bound_method = types.MethodType(standalone_function, obj)
print(f"绑定后类型: {type(bound_method)}")  # <class 'method'>

# 调用对比
print(f"直接调用函数: {standalone_function(obj, 'data1')}")
print(f"调用绑定方法: {bound_method('data2')}")  # 自动传递 obj
```
### Method 对象的内部结构
```python
import types

def my_function(self, value):
    return f"函数处理: {value}"

class TestClass:
    def __init__(self, name):
        self.name = name

obj = TestClass("示例")
bound_method = types.MethodType(my_function, obj)

print("=== Method 对象的内部结构 ===")
print(f"__self__ (绑定的对象): {bound_method.__self__}")    # __self__ (绑定的对象): <__main__.TestClass object at 0x0000017D03647FD0>
print(f"__func__ (原始函数): {bound_method.__func__}")  # __func__ (原始函数): <function my_function at 0x0000017D0366A440>
print(f"__class__ (对象类型): {bound_method.__class__}")    # __class__ (对象类型): <class 'method'>

# 验证绑定关系
print(f"绑定的对象是否是 obj: {bound_method.__self__ is obj}")  # 绑定的对象是否是 obj: True
print(f"原始函数是否是 my_function: {bound_method.__func__ is my_function}")    # 原始函数是否是 my_function: True
```
### 不同绑定类型的对比
```python
import types

def test_function(cls_or_self, data):
    if isinstance(cls_or_self, type):
        return f"类方法处理: {data} (类: {cls_or_self.__name__})"
    else:
        return f"实例方法处理: {data} (实例: {cls_or_self})"

class MyClass:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"MyClass({self.name})"

obj = MyClass("测试")

print("=== 不同绑定类型 ===")

# 1. 绑定到实例 (实例方法)
instance_method = types.MethodType(test_function, obj)
print(f"实例绑定: {type(instance_method)}")  # 实例绑定: <class 'method'>
print(f"调用结果: {instance_method('数据1')}") # 调用结果: 实例方法处理: 数据1 (实例: MyClass(测试))

# 2. 绑定到类 (类方法)
class_method = types.MethodType(test_function, MyClass) 
print(f"类绑定: {type(class_method)}")  ## 类绑定: <class 'method'>
print(f"调用结果: {class_method('数据2')}") # 调用结果: 类方法处理: 数据2 (类: MyClass)
```
### Method 对象的特殊行为
```python
import types

def original_function(self, x, y=10):
    return f"计算: {x} + {y} = {x + y}"

class Calculator:
    def __init__(self, name):
        self.name = name

calc = Calculator("我的计算器")
bound_method = types.MethodType(original_function, calc)

print("=== Method 对象的特殊行为 ===")

# 1. 自动参数传递
print("参数传递:")
print(f"原函数需要3个参数: original_function(self, x, y=10)")
print(f"绑定方法只需要2个参数: bound_method(x, y=10)")
print(f"调用结果: {bound_method(5)}")  # 调用结果: 计算: 5 + 10 = 15

# 2. 方法的可调用性
print(f"\n可调用性检查:")
print(f"bound_method 是否可调用: {callable(bound_method)}") # bound_method 是否可调用: True

# 3. 方法的字符串表示
print(f"\n字符串表示:")
print(f"bound_method: {bound_method}") # bound_method: <bound method original_function of <__main__.Calculator object at 0x000001E780B87E80>> 
print(f"repr: {repr(bound_method)}") # repr: <bound method original_function of <__main__.Calculator object at 0x000001E780B87E80>>
```
### 与描述符协议的关系
```python
import types

class MethodDescriptor:
    """演示 MethodType 如何与描述符协议配合"""
    
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner=None):
        if instance is None:
            return self.func  # 从类访问返回函数
        # 从实例访问返回绑定方法 (这就是 MethodType 的作用)
        return types.MethodType(self.func, instance)

def external_function(self, message):
    return f"{self.__class__.__name__}: {message}"

class TestClass:
    # 手动使用描述符
    manual_method = MethodDescriptor(external_function)
    
    def __init__(self, name):
        self.name = name

obj = TestClass("测试")

print("=== 描述符协议中的 MethodType ===")
print(f"从类访问: {type(TestClass.manual_method)}")  # 从类访问: <class 'function'>
print(f"从实例访问: {type(obj.manual_method)}")      # 从实例访问: <class 'method'>
print(f"调用结果: {obj.manual_method('Hello')}")    # 调用结果: TestClass: Hello
```


## class_or_instancemethod 的分析
```python
class class_or_instancemethod(classmethod):
    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, owner)
```
根据 Python 官方文档，`classmethod` 的纯Python等价实现是：
```python
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.f), '__get__'):
            # 将类对象绑定为第一个参数
            return self.f.__get__(cls, type(cls))
        return MethodType(self.f, cls)
```
即：`classmethod.__get__` 总是返回一个绑定到类对象的方法。无论是从类还是实例访问，第一个参数都是类对象（cls）。

而 `self.__func__` 是 `classmethod` 对象内部存储的原始函数。
